In [1]:
import sys
sys.path.append("..")

import time
import numpy as np 
import pandas as pd 

import blot
from dateutil import parser
from datetime import datetime

In [2]:
data = pd.read_csv("../data/storage.csv")
data["time"] = data.time.apply(lambda ts: datetime.fromtimestamp(ts))
# data = data.set_index(["time"])
data = data[(data["time"] > datetime(2020, 8, 30)) & (data["time"] < datetime(2020, 9, 20))]
data.head()

,time,value
3168,2020-09-11 00:00:00,1015
3169,2020-09-11 00:05:00,833
3170,2020-09-11 00:10:00,762
3171,2020-09-11 00:15:00,570
3172,2020-09-11 00:20:00,586


In [3]:
data.blot(x='time', y='value', dimensions=(1400, 900), colors={'value': 'blue'})

In [4]:
from troubleshooter.anomaly.tseries.sr import SpectralResidual
model = SpectralResidual(series=data, score_window=25, mag_window=3, threshold=3)
output = model.detect()

In [5]:
output.head()

,time,value,map,anomalyScore,isAnomaly
0,2020-09-11 00:00:00,1015,0.014989,0.000000,False
1,2020-09-11 00:05:00,833,0.033112,0.376766,False
2,2020-09-11 00:10:00,762,0.005561,0.689104,False
3,2020-09-11 00:15:00,570,0.070153,1.266384,False
4,2020-09-11 00:20:00,586,0.028151,0.073770,False


In [6]:
output["value"] = output["value"] / 1600.0
fig = output.blot(x='time', y=['value', 'map'], dimensions=(1400, 900), colors={'value': 'blue', 'map': 'green'}) + (output[output['isAnomaly']][['time','value']].rename(columns={"value": "anomaly"}).blot(x='time',y='anomaly', kind="scatter"))
fig.show()

In [7]:
output[output["isAnomaly"]==True]
with open("example.html", "w") as f:
    f.write(fig.to_html())